In [ ]:
import pandas as pd

In [ ]:
def determine_grade(scores):
    if scores >= 93 and scores <= 100:
        return 'A'
    elif scores >= 90 and scores < 93:
        return 'A-'
    elif scores >= 87 and scores < 90:
        return 'B+'
    elif scores >= 83 and scores < 87:
        return 'B'
    elif scores >= 80 and scores < 83:
        return 'B-'
    elif scores >= 77 and scores < 80:
        return 'C+'
    elif scores >= 73 and scores < 77:
        return 'C'
    elif scores >= 70 and scores < 73:
        return 'C-'
    elif scores >= 67 and scores < 70:
        return 'D+'
    elif scores >= 63 and scores < 67:
        return 'D'
    elif scores >= 60 and scores < 63:
        return 'D-'
    else:
        return 'F'

# Some where in the data processing up here I should combine all the sections and assign a column with section name and assign a point total to each assignment

# When I do this, I need to figure out what to do with the honors work, maybe include a line that splits the honors info or adds a y/n honors column but then what do I do with non-honors students in the blank columns because some honors students didn't submit work so I would need an if for that sit.

So give a list of the things
loop over them to add the section number and honors information
then if there is an honors section

In [ ]:
sect = ['01H','02','03','04','05']
dropColumns = ['Availability','Weighted Total','Total [Total','Safety Contract']

sectdl = []
for j in sect:
    exec('data' + j + "= pd.read_csv('./exampledata/CEM153_'+j+'_Stripped.csv')")
    exec('data' + j + "['Section']=j")
    if 'H' in j:
        exec('data' + j + "['Honors']='Y'")
    else:
        exec('data' + j + "['Honors']='N'")
    sectdl.append()

# lodf = []
# for j in sect:
#     temp = pd.read_csv('./exampledata/CEM153_'+j+'_Stripped.csv')
#     matching = [s for s in temp.columns if any(xs in s for xs in dropColumns)]
#     temp = temp.drop(columns=matching)
#     collist = []
#     for k in temp.columns:
#         end = k.find(']')
#         collist.append(k[0:end+1])
#     temp.columns=collist
#     print(collist)
#     lodf.append(temp)
#data = pd.concat(lodf,ignore_index=True,sort=False,axis=0)
#print(data)
#temp = pd.read_csv('./exampledata/CEM153_'+j+'_Stripped.csv')
#data = pd.read_csv('./exampledata/CEM153_03_Stripped.csv')
# deadColumnsBool = data.isnull().values.all(axis=0)
# deadColumns = []
# for i in range(len(deadColumnsBool)):
#     if deadColumnsBool[i] == True:
#         ind = data.columns[i]
#         deadColumns.append(ind)
# data = data.drop(columns=deadColumns)
# data = data.set_index(['Username'])
# data = data[:].fillna('No Submission')

In [ ]:
sectdl

# Should I be using a dictionary instead of a pandas dataframe? 

# When I eventually load in all the sections for one class, I am going to need to compile a master list of students and their sections and stuff

# I think I want to make a schedule to mark the dates of the assignments so I can attach due dates to them, but that's wishful thinking

# When the files get imported and I bring in 5 files at onces, when I bring them in I should make sure there is a new column added with the section number.

# Right now I have everything organized by student but may need to change this to think about things by section. I feel like this is the same problem I had with my research in excel this summer so I gotta figure out how to transform or slice the data for the assignments section because I'll need info from each section for each assignment to be compiled. Idk if just assigning a section number as a column is the most efficient way to do this if I have loops on loops.

# I need an outside program to call my classes that can choose to do one student, students by section, or all students and same for assignments and all the combinations of the two. Something with flags/options and argparse

In [ ]:
### STUDENT ###
## If you give first/last name it will pull up a list of names to then select from
# Student Name (F&L), username, student id, Section Number**********<- need to add this
# Attendance: *gives percentage*
# Calculated Grade including Missing Assignments as 0's: *gives grade as %, letter*
# Calculated Grade excluding Missing Assignments: *gives grades as %, letter*
# Section Average: *gives section average grade as %, letter* <- going to have to think about this in terms of data transformation
# Course Average: *gives class average grade as %, letter*
# Missing Assignments: *gives list like: assignment 1, assignment 2, ...* (each item on separate line)

In [ ]:
### Included in class student ###
# Student Name                  as fname & lname
# username                      as uname
# student id                    as stuid
# section number                as ADD THIS
# attendance %                  as attendance
# allgrades %                   as allgradesp
# allgrades letter              as allgradesl
# onlygraded %                  as onlygradedp
# onlygraded letter             as onlygradedl
# list of missing work          as missingwork
# list of needs to be graded    as needsgrading

### To be included in outside loops ###
# List of potentially matching students or list of students & user names
# Section Average Grade
# Course Average Grade

In [ ]:
class student:
    def __init__(self,data,username):
        self.uname = username
        self.fname = data.at[username,'First Name']
        self.lname = data.at[username,'Last Name']
        self.stuid = data.at[username,'Student ID']
        # Need to add section number
        self.attendance = data.at[username,data.filter(like='Attendance').columns[0]]
        self.studata = data.loc[[username]]
        
        dropColumns = ['Attendance','Last Name','First Name','Student ID','Last Access']
        matching = [s for s in data.columns if any(xs in s for xs in dropColumns)]
        data2 = data.drop(columns=matching)
        stot = 0
        allgrades = 0
        onlygraded = 0
        mwork = []
        ngrade = []
        for i in range(len(data2.columns)):
            colname = data2.columns[i]
            start = colname.find('[')
            end = colname.find(']')
            allgrades+=float(colname[start+12:end-6])
            if data.at[self.uname,colname] == 'Needs Grading' or data.at[self.uname,colname] == 'No Submission':
                stot+=0
                onlygraded+=0
                if data.at[self.uname,colname] == 'Needs Grading':
                    colname = colname[0:start]
                    ngrade.append(colname)
                elif data.at[self.uname,colname] == 'No Submission':
                    colname = colname[0:start]
                    mwork.append(colname)
            else:
                onlygraded+=float(colname[start+12:end-6])
                stot+=float(data.at[self.uname,colname])
        self.allgradesp = (stot/allgrades)*100
        self.allgradesl = determine_grade(self.allgradesp)
        self.onlygradedp = (stot/onlygraded)*100
        self.onlygradedl = determine_grade(self.onlygradedp)
        if mwork == []:
            mwork = ['You have no work that is missing']
        if ngrade == []:
            ngrade = ['You have no work that needs grading']
        self.missingwork = mwork
        self.needsgrading = ngrade

In [ ]:
dooku = student(data,'cdooku')
jocasta = student(data,'jnu')
tyranus = student(data,'dtyranus')
tyranus.needsgrading

In [ ]:
### ASSIGNMENT ###
## Gives a list of assignments to choose from if necessary 
# Assignment Name, points possible
# Calculated Avg for each section without 0's or missing assignmnets or 'Needs Grading'
# Calculated Avg for all sections ''
# Breakdown of who hasn't turned this in by section
# Breakdown of how many per section need to be graded
# Plot of section grades with course average as a line

In [ ]:
class assignment:
    def __init__(self,data,assignname):
        self.name = assignname
        self.adata = data[assignname]
        
        start = self.name.find('[')
        end = self.name.find(']')
        self.points = float(self.name[start+12:end-6])
        
        nstu = 0
        tot = 0
        for i in self.adata:
            if i == 'Needs Grading':
                continue
            elif i == 'No Submission':
                continue
            elif i == 0:
                continue
            else:
                nstu+=1
                tot+=i
        
        # Calculated Avg for each section without 0's or missing assignmnets or 'Needs Grading'
        # Calculated Avg for all sections ''
        # Breakdown of who hasn't turned this in by section
        # Breakdown of how many per section need to be graded

In [ ]:
lab1 = assignment(data,'L9 - Final Report [Total Pts: 100 Score] |44601') 

In [ ]:
dat = lab1.adata

In [ ]:
nstu = 0
tot = 0
for i in dat:
    print(i)
    if i == 'Needs Grading':
        continue
    elif i == 'No Submission':
        continue
    elif i == 0:
        continue
    else:
        nstu+=1
        tot+=i

# Should the course just be done in the outside code and not be a class because all the information I will need is stored in student and in assignment

In [ ]:
### COURSE ###
# Calculated Section Average
# Calculated Grade for All Sections
# Breakdown of Grades by assignment (section and class averages given)
# Compare sections with anova table
# Plots?